Data_Manipulation_Data_Cleaning
A Python project using Pandas and regular expressions to clean a dataset from the British Library, focusing on columns like 'Date of Publication,' 'Author,' 'Title,' and 'Place of Publication.' The result: a pristine dataset primed for analysis and visualization.

1. Read the csv in a data_frame.

In [1]:
import pandas as pd 
df = pd.read_csv(r'C:\Users\Dell\Desktop\Tutorials\Vinutha-Vijayakumar\Github\Data-Manipulation-Cleaning\Book.csv')


In [2]:
print(df.head(5))

   Identifier             Edition Statement      Place of Publication   
0         206                           NaN                    London  \
1         216                           NaN  London; Virtue & Yorston   
2         218                           NaN                    London   
3         472                           NaN                    London   
4         480  A new edition, revised, etc.                    London   

  Date of Publication              Publisher   
0         1879 [1878]       S. Tinsley & Co.  \
1                1868           Virtue & Co.   
2                1869  Bradbury, Evans & Co.   
3                1851          James Darling   
4                1857   Wertheim & Macintosh   

                                               Title     Author   
0                  Walter Forbes. [A novel.] By A. A      A. A.  \
1  All for Greed. [A novel. The dedication signed...  A., A. A.   
2  Love the Avenger. By the author of “All for Gr...  A., A. A.   
3  W

2. Remove the following columns from the data_frame
'Edition Statement', 
'Corporate Author', 
'Corporate Contributors', 
'Former owner', 
'Engraver', 
'Contributors', 
'Issuance type', 
'Shelfmarks'

In [3]:
columns_to_remove = [
    'Edition Statement',
    'Corporate Author',
    'Corporate Contributors',
    'Former owner',
    'Engraver',
    'Contributors',
    'Issuance type',
    'Shelfmarks'
]
df.drop(columns=columns_to_remove, inplace=True)

In [4]:
print(df.head(5))

   Identifier      Place of Publication Date of Publication   
0         206                    London         1879 [1878]  \
1         216  London; Virtue & Yorston                1868   
2         218                    London                1869   
3         472                    London                1851   
4         480                    London                1857   

               Publisher                                              Title   
0       S. Tinsley & Co.                  Walter Forbes. [A novel.] By A. A  \
1           Virtue & Co.  All for Greed. [A novel. The dedication signed...   
2  Bradbury, Evans & Co.  Love the Avenger. By the author of “All for Gr...   
3          James Darling  Welsh Sketches, chiefly ecclesiastical, to the...   
4   Wertheim & Macintosh  [The World in which I live, and my place in it...   

      Author                                         Flickr URL  
0      A. A.  http://www.flickr.com/photos/britishlibrary/ta...  
1  A., A. A.  

3. Check the content of the column- 'Date of Publication' and define a function to clean the value 
 Examples: 
1879 [1878] to: 1879
[1858.] to 1858

In [5]:
import re
import pandas as pd
import numpy as np

def clean_date_of_publication(date):
    if pd.notna(date) and isinstance(date, str):
        # Remove any square brackets.
        date = date.replace("[", "").replace("]", "")

        # Remove any periods at the end of the string.
        date = date.rstrip(".")

        # If the date is in the format of "1879 [1878]", extract only the first year.
        match = re.search(r'\d{4}', date)
        if match:
            date = match.group()

    return date

# Apply the cleaning function to the 'Date of Publication' column
df['Date of Publication'] = df['Date of Publication'].apply(clean_date_of_publication)



In [6]:
print(df["Date of Publication"].head(50))

0     1879
1     1868
2     1869
3     1851
4     1857
5     1875
6     1872
7      NaN
8     1676
9     1679
10    1802
11    1859
12    1888
13    1839
14    1897
15    1865
16    1860
17    1873
18    1866
19    1899
20    1814
21    1820
22    1800
23    1847
24    1897
25    1897
26    1893
27    1805
28    1837
29    1896
30    1898
31    1899
32    1892
33    1894
34    1894
35     NaN
36    1899
37    1885
38    1860
39    1846
40    1893
41    1894
42    1817
43    1816
44    1817
45    1897
46    1833
47    1804
48    1859
49    1777
Name: Date of Publication, dtype: object


4. Check the content of the column- ' Author' and define a function to clean the value. And split the name to first name and last name.

In [7]:
import pandas as pd
import re

# Define a function to clean and split the author names
def clean_and_split_author_name(author):
    if isinstance(author, str):
        # Remove any special characters, punctuation, and extra spaces
        cleaned_author = re.sub(r'[^a-zA-Z\s]', '', author)

        # Split the cleaned name into words
        words = cleaned_author.split()

        # Handle cases where there are multiple words in the author name
        if len(words) >= 2:
            first_name = words[0]
            last_name = ' '.join(words[1:])
        else:
            first_name = words[0]
            last_name = None

        return {'First Name': first_name, 'Last Name': last_name}
    else:
        return {'First Name': None, 'Last Name': None}

# Apply the cleaning and splitting function to the "Author" column
df['Author Info'] = df['Author'].apply(clean_and_split_author_name)

# Expand the dictionary into separate "First Name" and "Last Name" columns
df[['First Name', 'Last Name']] = pd.DataFrame(df['Author Info'].tolist())



In [8]:
print(df[["Author", "First Name", "Last Name"]].head(20))


                                               Author First Name   
0                                               A. A.          A  \
1                                           A., A. A.          A   
2                                           A., A. A.          A   
3                                           A., E. S.          A   
4                                           A., E. S.          A   
5                                           A., E. S.          A   
6                                           A., F. E.          A   
7                                       A., J.|A., J.          A   
8                                              Remaʿ.       Rema   
9                                              A., T.          A   
10                                                NaN       None   
11                                       AALL, Jacob.       AALL   
12  AAR, Ermanno - pseud. [i.e. Luigi Giuseppe Oro...        AAR   
13                                              

5. Check the content of the column- 'Title' and define a function to clean the value .
Examples:
 Walter Forbes. [A novel.] By A. A to: Walter Forbes
 Love the Avenger. By the author of 'All for Gr.. to Love The Avenger 

In [9]:
import pandas as pd

# Define a function to clean the "Title" column
def clean_title(title):
    # Remove square brackets without removing content inside them
    title = title.replace('[', '').replace(']', '')
    
    # Find the position of the first occurrence of a character
    # other than letters, numbers, spaces, or hyphens
    match = re.search(r'[^a-zA-Z0-9\s\-]', title)
    
    if match:
        # Extract the substring before the first occurrence of that character
        title = title[:match.start()]
    
    # Remove single quotes and extra spaces
    title = title.strip().replace('.', '').replace("'", "")
    
    return title

# Apply the cleaning function to the "Title" column
df['Title'] = df['Title'].apply(clean_title)



In [10]:
# Display the cleaned DataFrame
print(df['Title'].head(20))

0                                         Walter Forbes
1                                         All for Greed
2                                      Love the Avenger
3                                        Welsh Sketches
4                             The World in which I live
5                             The World in which I live
6                                             Lagonells
7                                  The Coming of Spring
8               A Warning to the inhabitants of England
9                                A Satyr against Vertue
10               An Account of the many and great Loans
11    Erindringer som Bidrag til Norges Historie fra...
12                         Gli Studi storici in terra d
13                                           De Aardbol
14                   Cronache Savonesi dal 1500 al 1570
15                                              See-Saw
16                                                   Ge
17                                     With elev

6. Check the content of the column- 'Place of Publication' and define a function to clean the value. E.g., the original value: London; Virtue & Yorston to: London.

In [11]:
import pandas as pd

# Define a function to clean the "Place of Publication" column
def clean_place_of_publication(place):
    # Split the string by semicolon, comma, or square bracket and keep the first part
    parts = re.split(r'[;,]', place)
    cleaned_place = parts[0].strip()
    
    return cleaned_place

# Apply the cleaning function to the "Place of Publication" column
df['Place of Publication'] = df['Place of Publication'].apply(clean_place_of_publication)

# Display the cleaned DataFrame
print(df['Place of Publication'])


0                    London
1                    London
2                    London
3                    London
4                    London
               ...         
8282                 London
8283                  Derby
8284                 London
8285    Newcastle upon Tyne
8286                 London
Name: Place of Publication, Length: 8287, dtype: object
